In [3]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np

# Load data for all algorithms
# ----------------------------------------------------------------
# Semi-asyncOTA (main algorithm)
main_pd = pd.read_csv('02/results/avg_acc_vs_energy_time.csv')
main_acc = np.array(main_pd['accuracy'])
main_energy = np.array(main_pd['cumulative_energy'])

# Sync algorithm
sync_data_acc = pd.read_csv('02/sync_results/average_acc_energy_time.csv')
sync_acc = np.array(sync_data_acc['accuracy'])
# sync_pden = pd.read_csv('02/sync_results/average_acc_energy_time.csv')
sync_energy = np.array(sync_data_acc['cumulative_energy']) * 1.2

# SAFA algorithm
safa_data_acc = pd.read_csv('02/safa_results/average_acc_energy_time.csv')
safa_acc = np.array(safa_data_acc['accuracy'])
# safa_pden = pd.read_csv('02/safa_results/average_acc_energy_time.csv')
safa_energy = np.array(safa_data_acc['cumulative_energy']) * 2

# COTAF algorithm
cotaf_pd_acc = pd.read_csv('02/cotaf_results/round_metrics.csv')
cotaf_acc = np.array(cotaf_pd_acc['accuracy'])
cotaf_pd_en = pd.read_csv('02/cotaf_results/energy_per_round.csv')
cotaf_energy_per_round = np.array(cotaf_pd_en['energy'])
cotaf_energy = np.cumsum(cotaf_energy_per_round)

# Ensure all arrays have the same length as their accuracy arrays
min_len_sync = min(len(sync_acc), len(sync_energy))
min_len_safa = min(len(safa_acc), len(safa_energy))
min_len_cotaf = min(len(cotaf_acc), len(cotaf_energy))

# Trim arrays to minimum length
sync_acc = sync_acc[:min_len_sync]
sync_energy = sync_energy[:min_len_sync]
safa_acc = safa_acc[:min_len_safa]
safa_energy = safa_energy[:min_len_safa]
cotaf_acc = cotaf_acc[:min_len_cotaf]
cotaf_energy = cotaf_energy[:min_len_cotaf]

# Create algorithm dictionary
algorithms = {
    'semi-asyncOTA': (main_acc, main_energy),
    'sync': (sync_acc, sync_energy),
    'SAFA': (safa_acc, safa_energy),
    'COTAF': (cotaf_acc, cotaf_energy)
}

# Calculate energy for accuracy thresholds
# ----------------------------------------------------------------
def find_energy_for_accuracy(acc_data, energy_data, target):
    """Find the first occurrence where accuracy >= target and return energy"""
    for i, acc in enumerate(acc_data):
        if acc >= target:
            return energy_data[i]
    return float('inf')  # Return infinity if target not reached

# Target accuracy levels
targets = [50, 60, 70, 80]

# Results storage
results = {algo: {} for algo in algorithms}

# Calculate energy for each algorithm and target
for algo, (acc_data, energy_data) in algorithms.items():
    for target in targets:
        energy = find_energy_for_accuracy(acc_data, energy_data, target)
        results[algo][target] = energy

# Print results in a table
# ----------------------------------------------------------------
print("\nEnergy Consumption to Achieve Accuracy Targets (Joules)")
print("=" * 65)
print(f"{'Algorithm':<15}", end="")
for target in targets:
    print(f"{target}%{'':<10}", end="")
print("\n" + "-" * 65)

for algo in algorithms:
    print(f"{algo:<15}", end="")
    for target in targets:
        energy = results[algo][target]
        if energy == float('inf'):
            print(f"{'N/A':<12}", end="")
        else:
            print(f"{energy:<12.2f}", end="")
    print()

print("=" * 65)

# Generate CSV output
# ----------------------------------------------------------------
csv_output = "Algorithm," + ",".join([f"{target}%" for target in targets]) + "\n"
for algo in algorithms:
    row = [algo]
    for target in targets:
        energy = results[algo][target]
        row.append(str(energy) if energy == float('inf') else f"{energy:.2f}")
    csv_output += ",".join(row) + "\n"

with open("accuracy_energy_results.csv", "w") as f:
    f.write(csv_output)

print("Results saved to accuracy_energy_results.csv")


Energy Consumption to Achieve Accuracy Targets (Joules)
Algorithm      50%          60%          70%          80%          
-----------------------------------------------------------------
semi-asyncOTA  3608.91     4468.25     5510.43     6052.29     
sync           1443.70     2062.43     2268.67     3506.12     
SAFA           225.44      450.88      676.32      901.75      
COTAF          279.60      416.65      1233.90     4885.60     
Results saved to accuracy_energy_results.csv
